# 文本分类

文本分类 是一种常见的自然语言处理（NLP）任务，它的作用是为文本分配标签或类别。很多大公司都在实际应用中使用文本分类，比如判断用户评论是好评还是差评。其中，情感分析 是最流行的文本分类形式之一，它会给文本打上“积极”、“消极”或“中立”的标签。

这本指南会教你怎么做两件事：
1. 微调 DistilBERT：在 IMDb 数据集上训练模型，让它能判断电影评论是好评还是差评。
2. 使用微调模型进行推理：用训练好的模型来分析新的评论。

要查看与此任务兼容的所有架构和 checkpoints，[可查阅](https://huggingface.co/tasks/text-classification)。

在开始之前，请确保安装了所有必要的库：

In [ ]:
pip install transformers datasets evaluate accelerate

在这一步，需要登录您的 Hugging Face 帐户，出现相关的登陆提示时，输入您的令牌以登录：

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

##  加载 IMDB 数据集

首先，从 IMDB 数据库中加载 IMDB 数据集：

In [ ]:
from datasets import load_dataset

imdb = load_dataset("imdb")

再看一个例子：

In [ ]:
imdb["test"][0]

此数据集中有两个字段：
- `text`：电影评论的文本。
- `label`：一个整数值，0表示负面评价，1表示正面评价。

## 预处理数据

下一步是加载一个 `DistilBERT Tokenizer` 来预处理`text`字段：

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

创建一个预处理函数来标记文本并截断序列，使其不超过 DistilBERT 的最大输入长度：

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

若要对整个数据集应用预处理函数，请使用函数 `[Datasets.map()](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.map)`。您可以通过设置`batched=True`来一次性处理数据集的多个元素来加速 `map`：

In [ ]:
tokenized_imdb = imdb.map(preprocess_function, batched=True)

现在使用[DataCollatorWithPadding](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorWithPadding)创建一批示例。

在排序过程中，会动态地将句子填充到与这一批数据中最长序列相同的长度，而不是将整个数据集都填充到最大长度，这样会更有效率。

In [ ]:
# PyTorch
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# TensorFlow
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

## 评估

在训练模型时，选择一个合适的指标来评估模型的表现非常重要。

你可以使用“[Evaluate](https://huggingface.co/docs/evaluate/index)”这个工具库，轻松地加载和使用各种评估方法。对于这个任务，我们需要加载一个叫做“[accuracy 准确度](https://huggingface.co/spaces/evaluate-metric/accuracy)”的度量指标。

你可以参考[Evaluate 快速指南](https://huggingface.co/docs/evaluate/a_quick_tour)，那里有详细的步骤教你如何加载和计算这些指标。

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

然后，创建一个函数，将你的预测和标签传递给 [computing](https://huggingface.co/docs/evaluate/main/en/package_reference/main_classes#evaluate.EvaluationModule.compute) 来计算准确度：

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

`compute_metrics`这个函数现在已经准备好了。当你开始设置训练过程时，你会再次用到它。

## 训练

在开始训练模型之前，我们需要用`id2label`和`label2id`来建立一个映射关系，这样就能把模型的输出编号转换成我们看得懂的标签了。

In [ ]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

### PyTorch

如果您不熟悉如何使用`[Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer)`对模型进行微调，请查看[此处](../tutorials/5_fine_tune_pretrained_model.ipynb)的基本教程！

现在你可以开始训练你的模型啦！使用AutoModelForSequenceClassification来加载DistilBERT模型，同时指定你需要的标签数量和标签映射关系：

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

此时，只剩下三个步骤：

1. 设置训练参数：在`TrainingArguments`中定义训练的超参数。这里最重要的是`output_dir`参数，它用来指定保存模型的位置。如果你想把模型上传到Hugging Face，需要设置`push_to_hub=True`（记得先登录Hugging Face账号）。在每个训练周期（epoch）结束时，`Trainer`会自动评估模型的准确性，并保存训练的检查点。
2. 配置`Trainer`：把训练参数、模型、数据集、分词器（tokenizer）、数据整理器（data collator）以及`compute_metrics`函数一起传递给 Trainer。
3. 开始训练：调用`train()`方法来微调你的模型。

In [ ]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

**当你传递`tokenizer`给`Trainer`的时候，它会默认使用动态填充功能。在这种情况下，你不需要手动地指定一个数据整理器。**

训练完成后，可以使用 `[push_to_hub()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.push_to_hub)` 方法将您的模型共享到 Hub，以便每个人都可以使用您的模型：

In [ ]:
trainer.push_to_hub() 

### TensorFlow

如果您不熟悉使用 Keras 微调模型，请查看[此处](../tutorials/5_fine_tune_pretrained_model.ipynb)的基本教程！

要在TensorFlow中微调模型，首先要设置优化器函数、学习率计划和一些训练超参数：

In [ ]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 16
num_epochs = 5
batches_per_epoch = len(tokenized_imdb["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

接下来，你可以使用`TFAutoModelForSequenceClassification`来加载`DistilBERT`模型，同时指定预期的标签数量和标签映射关系：

In [ ]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

使用`[prepare_tf_dataset()](https://huggingface.co/docs/transformers/main/en/main_classes/model#transformers.TFPreTrainedModel.prepare_tf_dataset)`将数据集转换为`tf.data.Dataset`格式：

In [ ]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_imdb["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_imdb["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

使用[compile](https://keras.io/api/models/model_training_apis/#compile-method)方法来配置模型进行训练。需要注意的是，Transformers库中的模型已经内置了与任务相关的默认损失函数，所以通常情况下你不需要手动指定损失函数，除非你有特别的需要：

In [ ]:
import tensorflow as tf

model.compile(optimizer=optimizer)  # 没有指定损失函数

在开始训练之前，还需要完成最后两个步骤：
1. 计算预测的准确性：我们需要评估模型预测的准确性。
2. 提供模型上传到Hub的方法：这样可以将训练好的模型分享到Hugging Face的Hub。

这两个步骤都可以通过使用[Keras回调](https://huggingface.co/docs/transformers/main/en/main_classes/keras_callbacks)来实现。

具体来说，你可以将`compute_metrics`函数传递给`[KerasMetricCallback](https://huggingface.co/docs/transformers/main/en/main_classes/keras_callbacks#transformers.KerasMetricCallback)`：

In [ ]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

在[PushToHubCallback](https://huggingface.co/docs/transformers/main/en/main_classes/keras_callbacks#transformers.PushToHubCallback)中指定将模型和 tokenizer 推送到何处：

In [ ]:
from transformers.keras_callbacks import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    output_dir="my_awesome_model",
    tokenizer=tokenizer,
)

接下来，把你的回调函数组合在一起：

In [ ]:
callbacks = [metric_callback, push_to_hub_callback]

最后，可以开始训练你的模型啦！调用[fit](https://keras.io/api/models/model_training_apis/#fit-method)方法，传入你的训练和验证数据集、训练周期（epoch）的数量以及你设置的回调函数，这样就能微调你的模型了：

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=callbacks)

训练完成后，您的模型将自动上传到Hub，以便每个人都可以使用它！

## 推理

当你已经微调了一个模型，你就可以用它来推理了！

抓取一些你想运行推理的文本：

In [ ]:
text = "This was a masterpiece. Not completely faithful to the books, but enthralling from beginning to end. Might be my favorite of the three."

要尝试使用微调后的模型进行推理，最简单的方法是通过 pipeline 来操作。你可以用你的模型创建一个专门用于情感分析的 pipeline，然后把你的文本输入进去：

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="stevhliu/my_awesome_model")
classifier(text)

如果有需要，您还可以手动复制 pipeline 的结果：

### PyTorch

1. 标记文本并返回PyTorch张量：对输入文本进行分词处理，并将其转换为PyTorch张量格式。

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("stevhliu/my_awesome_model")
inputs = tokenizer(text, return_tensors="pt")

2. 将输入传递给模型并返回logits：把处理好的张量输入到模型中，获取模型的输出`logits`。

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("stevhliu/my_awesome_model")
with torch.no_grad():
    logits = model(**inputs).logits

3. 获取概率最高的类别，并使用模型的`id2label`映射将其转换为文本标签：从`logits`中找出概率最高的类别，然后利用模型的`id2label`映射关系，将其转换成对应的文本标签。

In [ ]:
predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

### TensorFlow

1. 标记文本并返回PyTorch张量：对输入文本进行分词处理，并将其转换为PyTorch张量格式。

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("stevhliu/my_awesome_model")
inputs = tokenizer(text, return_tensors="tf")

2. 将输入传递给模型并返回logits：把处理好的张量输入到模型中，获取模型的输出`logits`。

In [ ]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("stevhliu/my_awesome_model")
logits = model(**inputs).logits

3. 获取概率最高的类别，并使用模型的`id2label`映射将其转换为文本标签：从`logits`中找出概率最高的类别，然后利用模型的`id2label`映射关系，将其转换成对应的文本标签。

In [ ]:
predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])
model.config.id2label[predicted_class_id]